In [1]:
#!pip install pytelegrambotapi

In [2]:
import telebot
from telebot import types
import requests
import random
import re

In [3]:
bot = telebot.TeleBot('')  # enter token

In [4]:
command_list = """\
    /start - Приветствие,
    /exchange - Курс основных валют,
    /words - Начать игру \"Угадай слово\",
    /stopgame - Закончить игру \"Угадай слово\"
"""

valutes = {
    'USD': 'Доллар США',
    'EUR': 'Евро',
    'GBP': "Фунт Стерлингов",
    'TRY': "Турецкая лира",
    'JPY': "Йена",
}

words = [
    'любознательность',
    'телескоп',
    'метрополитен',
    'энергетика',
    'невероятный',
    'преобразовывать',
    'импульсивный',
    'короткометражный',
    'амбивалентность',
    'предсказание'
]

# key - user id, value - dict
# value contains 3 items: word(string) - given word, mask(list) - for progress output, game_on(bool) - flag
game_data = {}

In [5]:
def get_exchange_rate(valute):
    response = requests.get("https://www.cbr-xml-daily.ru/daily_json.js")
    result = ""
                            
    if response.status_code == 200:
        response_json = response.json()
        exchange_rate = response_json['Valute'][valute]['Value']
        result = '1 {0} = {1} руб.'.format(valute, round(exchange_rate, 2))

    else:
        result = "request to CBR failed"

    return result

def create_keyboard_for_exchange_func():
    keyboard = types.InlineKeyboardMarkup()
                            
    for key, value in valutes.items():
        button = types.InlineKeyboardButton(text=value, callback_data='exchange_' + key)
        keyboard.add(button)
    
    return keyboard

def get_word_mask(word):
    mask = []
                            
    for _ in range(len(word)):
        mask.append('*')

    return mask
                            
def guess_word(user_id, text):
    global game_data
    text = text.lower()
    letter_guessed = False

    if re.search(r'\b[а-я]{1}\b', text) is not None:
        for i in range (len(game_data[user_id]['word'])):
            if game_data[user_id]['word'][i] == text:
                letter_guessed = True
                game_data[user_id]['mask'][i] = game_data[user_id]['word'][i]

        if letter_guessed == False:
            bot.send_message(user_id, text='Увы, в загаданном слове нет буквы {0}! Ваше слово: {1}'.format(
                text, mask_to_string(game_data[user_id]['mask'])))

        elif re.search(r'[*]+', mask_to_string(game_data[user_id]['mask'])) is None:
            bot.send_message(user_id, text='Ура, Вы победили! Ваше слово: {0}'.format(game_data[user_id]['word']))
            game_data[user_id]['game_on'] = False

        elif letter_guessed == True:
            bot.send_message(user_id, text='Вы отгадали букву! Ваше слово: {0}'.format(
                mask_to_string(game_data[user_id]['mask'])))
    else:
        bot.send_message(user_id, text='Я не смог распознать символ. Пожалуйста, вводите по одной букве за раз.')

def mask_to_string(mask):
    return "".join(mask)

In [6]:
@bot.message_handler(content_types=['text'])
def handle_commands(command):
    global game_data
    user_id = command.from_user.id

    if command.text == '/help':
        bot.send_message(user_id, command_list)

    elif command.text == '/start':
        bot.send_message(user_id, 'Привет!')
        
    elif command.text == '/exchange':
        keyboard = create_keyboard_for_exchange_func()
        bot.send_message(user_id, text='Курс какой валюты Вы хотите узнать?', reply_markup=keyboard)

    elif command.text == '/words':
        word = words[random.randint(0, len(words)-1)]
        game_data[user_id] = {'word': word, 'mask': get_word_mask(word), 'game_on': True}
        bot.send_message(user_id, text='Игра началась! Называйте буквы по очереди, как в поле чудес. Хотите закончить игру - пишите /stopgame')
        bot.send_message(user_id, 'Ваше слово: {0}'.format(mask_to_string(game_data[user_id]['mask'])))

    elif command.text == '/stopgame':
        game_data[user_id] = {'word': '', 'mask': '', 'game_on': False}
        bot.send_message(user_id, 'Игра завершена. Возвращайтесь еще!')
        
    elif game_data[user_id] is not None:
        if game_data[user_id]['game_on'] == True:
            guess_word(user_id, command.text)

            
@bot.callback_query_handler(func=lambda call: True)
def handler_buttons(call):
    if call.data.startswith('exchange'):
        data_split = call.data.split('_')
        bot.send_message(call.message.chat.id, get_exchange_rate(data_split[1]))

In [7]:
# run bot
bot.polling(none_stop=True, interval=0)